In [1]:
!pip install numdifftools
!pip install corner
# importing stuff
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress, chi2
from astropy.io import fits
from scipy.integrate import quad
from scipy.optimize import curve_fit, minimize, approx_fprime
import numdifftools as nd
import csv
from matplotlib.patches import Ellipse
import corner

In [2]:
# mount drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# downloading data
csv_data = '/content/drive/MyDrive/2024project/DES-SN5YR_HD.csv'
df = pd.read_csv(csv_data)
z_obs = df['zHD']
mu_obs = df['MU']

sys_data = np.loadtxt('/content/drive/MyDrive/2024project/STAT+SYS_new.txt')
mu_error = df['MUERR_FINAL']

# reshaping stat and sys files into a square matrix
shape = (1829,1829)
sys_data_proper = sys_data.reshape(shape)

#add sys and stat
np.fill_diagonal(sys_data_proper, sys_data_proper.diagonal() + mu_error**2)

# inverting matrix
inverse_cm = np.linalg.inv(sys_data_proper)

c = 3e5 # km/s

print(len(z_obs))
print(len(mu_obs))

1829
1829


In [4]:
from scipy.stats import linregress
from astropy.io import fits
from scipy.integrate import quad
from scipy.optimize import curve_fit, minimize, approx_fprime
import csv

In [5]:
# minimising chi2 for Flat-ΛCDM
c = 3e5 # km/s

# flat-ΛCDM, fitting for omega_M using eqn from table 1 from DES paper
def E(z, OmegaM):
    hubble_param = np.sqrt(OmegaM * (1 + z)**3 + (1 - OmegaM))
    return hubble_param
    print(hubble_param)

# comoving distance, equation 2 from DES paper
def R0_chi(z, OmegaM, H0):
    def integrand(z):
        return 1 / E(z, OmegaM)

    # performing numerical integration
    integral, _ = quad(integrand, 0, z)
    return (c / H0) * integral

# theoretical distance modulus
def mu_theoretical(OmegaM, H0, z):
  dL = (1+z) * (R0_chi(z, OmegaM, H0)) # changed z_obs to z
  mu = 5* np.log10(dL) + 25 # theoretical mu
  return mu

# chi-squared function
def chi2(params):
  OmegaM, H0 = params
  mu_model = np.array([mu_theoretical(OmegaM, H0, z) for z in z_obs])

  dmu = mu_obs - mu_model # observed mu - theoretical mu
  # transpose of dmu
  dmu_T = dmu.transpose()
  return np.dot(dmu, np.dot(inverse_cm, dmu_T))

initial_guess = [0.3, 50.0]  # OmegaM, H0 (H0 in km/s/Mpc)
result = minimize(chi2, x0 = initial_guess, method='nelder-mead')

print("Optimisation Results:")
print(f"OmegaM = {result.x[0]:.4f}")
print(f"H0 = {result.x[1]:.4f} km/s/Mpc")
print(f"Minimum chi-squared = {result.fun:.2f}")

Optimisation Results:
OmegaM = 0.3499
H0 = 69.3691 km/s/Mpc
Minimum chi-squared = 1640.31


In [6]:
# Flat wCDM model

def E(z, OmegaM, w):
    return np.sqrt(OmegaM * (1 + z)**3 + (1 - OmegaM) * (1 + z)**(3 * (1 + w)))

# comoving distance
def R0_chi(z, OmegaM, w, H0):
    def integrand(z):
        return 1 / E(z, OmegaM, w)

    integral, _ = quad(integrand, 0, z)
    return (c / H0) * integral

# theoretical distance modulus
def mu_theoretical(OmegaM, w, H0, z):
    dL = (1 + z) * R0_chi(z, OmegaM, w, H0)
    return 5 * np.log10(dL) + 25

# chi-squared function
def chi2(params):
    OmegaM, w, H0 = params
    mu_model = np.array([mu_theoretical(OmegaM, w, H0, z) for z in z_obs])

    dmu = mu_obs - mu_model  # observed - theoretical
    dmu_T = dmu.transpose()
    return np.dot(dmu, np.dot(inverse_cm, dmu_T))

initial_guess = [0.3, -1.0, 50]  # OmegaM, w
result2 = minimize(chi2, x0=initial_guess, method='nelder-mead')

print("Optimisation Results:")
print(f"OmegaM = {result2.x[0]:.3f}")
print(f"w = {result2.x[1]:.4f}")
print(f"H0 = {result2.x[2]:.4f} km/s/Mpc")
print(f"Minimum chi-squared = {result2.fun:.2f}")

Optimisation Results:
OmegaM = 0.273
w = -0.8183
H0 = 69.0961 km/s/Mpc
Minimum chi-squared = 1639.00


In [7]:
# Flat-w0waCDM model

def E(z, OmegaM, w0, wa):
    exponent = -3 * wa * z / (1 + z)
    return np.sqrt(OmegaM * (1 + z)**3 + (1 - OmegaM) * (1 + z)**(3 * (1 + w0 + wa)) * np.exp(exponent))

# comoving distance
def R0_chi(z, OmegaM, w0, wa, H0):
    def integrand(z):
        return 1 / E(z, OmegaM, w0, wa)

    integral, _ = quad(integrand, 0, z)
    return (c / H0) * integral

# theoretical distance modulus
def mu_theoretical(OmegaM, w0, wa, H0, z):
    dL = (1 + z) * R0_chi(z, OmegaM, w0, wa, H0)
    return 5 * np.log10(dL) + 25

# chi-squared function
def chi2(params):
    OmegaM, w0, wa, H0 = params
    mu_model = np.array([mu_theoretical(OmegaM, w0, wa, H0, z) for z in z_obs])

    dmu = mu_obs - mu_model  # observed - theoretical
    dmu_T = dmu.transpose()
    return np.dot(dmu, np.dot(inverse_cm, dmu_T))

initial_guess = [0.3, -0.3, -8, 50.0]  # OmegaM, w0, wa, H0
result3 = minimize(chi2, x0=initial_guess, method='nelder-mead')

print("Optimisation Results:")
print(f"OmegaM = {result3.x[0]:.4f}")
print(f"w0 = {result3.x[1]:.4f}")
print(f"wa = {result3.x[2]:.4f}")
print(f"H0 = {result3.x[3]:.4f} km/s/Mpc")
print(f"Minimum chi-squared = {result3.fun:.2f}")

Optimisation Results:
OmegaM = 0.4865
w0 = -0.4322
wa = -7.9888
H0 = 68.2321 km/s/Mpc
Minimum chi-squared = 1632.70


In [8]:
import scipy
from scipy.stats import norm

# two tailed tests to find the p value and significance

# LCDM vs w0waCDM

delta_chi2 = result.fun - result3.fun # always do simpler model - complexer model
dof = 2 # difference in number of params between models

p_value = 1 - scipy.stats.chi2.cdf(delta_chi2, dof)

sigma = norm.ppf(1 - p_value/2)
print("Two-tailed p-value:", p_value)
print("Significance level (σ):", sigma)

Two-tailed p-value: 0.022299024155470026
Significance level (σ): 2.2852354745042107


In [9]:
# LCDM vs wCDM

delta_chi2 = result.fun - result2.fun # always do simpler model - complexer model
dof = 1 # difference in number of params between models

p_value = 1 - scipy.stats.chi2.cdf(delta_chi2, dof)

sigma = norm.ppf(1 - p_value/2)
print("Two-tailed p-value:", p_value)
print("Significance level (σ):", sigma)

Two-tailed p-value: 0.252885737372919
Significance level (σ): 1.1433682299009305


In [10]:
# wCDM vs w0waCDM

delta_chi2 = result2.fun - result3.fun # always do simpler model - complexer model
dof = 1 # difference in number of params between models

p_value = 1 - scipy.stats.chi2.cdf(delta_chi2, dof)

sigma = norm.ppf(1 - p_value/2)
print("Two-tailed p-value:", p_value)
print("Significance level (σ):", sigma)

Two-tailed p-value: 0.01207969833759348
Significance level (σ): 2.509807525187759
